# **MOUNTAIN CAR**

Import required modules

In [ ]:
import gym
from abc import ABC, abstractmethod
import random
import sys
import numpy as np
import matplotlib.pyplot as plt

Set to `True` if you want the agent learn from the environment, set to `False` if you want to test the knowledge aquired previously

In [ ]:
Training = True

Make the environment

In [ ]:
if Training == True:
    env = gym.make('MountainCar-v0')
else:
    env = gym.make('MountainCar-v0', render_mode="human")

Discretize the state space

In [ ]:
num_states = [20, 20]
num_actions = env.action_space.n
state_space = [np.linspace(env.observation_space.low[0], env.observation_space.high[0], num_states[0]), np.linspace(env.observation_space.low[1], env.observation_space.high[1], num_states[1])]

# Computing policies

Base class for all policies. The method <code>apply</code> enforces the policy.

In [ ]:
class BasePolicy(ABC):

    @abstractmethod
    # Ovveride this method to implement policy application
    # Returns the action given the state
    def apply(self, state):
        pass

A concrete policy to initialize the search for an optimal policy. Implements an $\epsilon$-greedy technique for action choice.

In [ ]:
class EpsilonGreedyPolicy(BasePolicy):
    def __init__(self, environment, Q_table, epsilon):
        self.environment = environment
        self.Q_table = Q_table
        self.epsilon = epsilon

    def apply(self, state):
        # Explore random action with probability epsilon
        if Training and random.random() < self.epsilon:
            return env.action_space.sample()

        #Choose the best action according to Q_table with probability 1-epsilon
        #If all actions have the same Q-value then break ties randomly
        else:
            max_action_value = -1 * sys.float_info.max

            for action in range(num_actions):
                if self.Q_table[state[0]][state[1]][action] > max_action_value:
                    max_action_value = self.Q_table[state[0]][state[1]][action]
                    best_action = action
            return best_action

# SARSA algorithm (on-policy)

For each episode, the agent chooses an action based on the epsilon-greedy policy (or greedy policy), executes the action, receives a reward and a new state, and then updates the Q-table based on the difference between the expected and actual reward.

In [ ]:
class SARSA:
    def __init__(self, environment, gamma, alpha, epsilon, episodes):
        self.environment = environment
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.episodes = episodes
        if Training:
            self.Q_table = np.zeros((21,21, num_actions))
        else:
            self.Q_table = np.load('Q_table.npy')

        self.policy = EpsilonGreedyPolicy(environment, self.Q_table, epsilon)

        self.rewards = np.zeros(self.episodes//100)

    #Return the indices of the bins to which each value in input array belongs.
    def digitizeState(self, state):
        new_state_p = np.digitize(state[0], state_space[0])
        new_state_v = np.digitize(state[1], state_space[1])
        return [new_state_p, new_state_v]


    def apply(self):
        i = 0
        total_reward = 0
        for episode in range(self.episodes):
            self.environment.reset()
            state,_ = self.environment.reset()
            state = self.digitizeState(state)
            action = self.policy.apply(state)
            done = False
            if episode % 1000 == 0:  
                print("Episode: ", episode)   
            while not done:
                nextstate, reward, done, tr, info = self.environment.step(action)
                nextstate = self.digitizeState(nextstate)
                next_action = self.policy.apply(nextstate)
                self.Q_table[state[0]][state[1]][action] += self.alpha * (reward + self.gamma * self.Q_table[nextstate[0]][nextstate[1]][next_action] - self.Q_table[state[0]][state[1]][action])
                state = nextstate
                action = next_action
                total_reward += reward
            if episode % 100 == 0:
                self.rewards[i] = total_reward
                total_reward = 0
                i += 1
                self.policy.epsilon *= 0.99
        
            self.environment.reset()
        np.save('Q_table.npy', self.Q_table)
        self.environment.close()

# Experimenting the SARSA algorithm

The function SARSA take as parameters:  

`environment`, that is the environment in which the agent plays,   

`alpha`, that is the learning rate of the algorithm. It determines how much the agent learns from each new experience,

`gamma`, that is the discount factor. Determines how much the agent cares about future rewards versus immediate rewards. 

`epsilon`, that is the parameter for the epsilon-greedy policy. Determine the probability that the agent will choose a random action over the action it thinks is best.  

`episodes`: This is the number of episodes the agent must play while learning.

In [ ]:
sarsa = SARSA(env, 0.8, 0.2, 0.3, 30000)
sarsa.apply()

# Plot

Plots the sum of rewards every 100 episodes

In [ ]:
x = np.arange(200, len(sarsa.rewards) * 100 + 1, 100)
z = np.polyfit(x, sarsa.rewards[1:], 1)
p = np.poly1d(z)
plt.plot(x, sarsa.rewards[1:])
plt.plot(x, p(x), "r--") 
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.title('Reward per 100 Episode')
plt.show()